###IMPORTING LIBRARIES

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import math
import sklearn
from sklearn import datasets, linear_model, metrics, svm, cluster
from sklearn.feature_extraction import DictVectorizer
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline

###READING DATA

In [ ]:
#loading dataset
master_df = pd.read_csv("../data/final.csv")

FileNotFoundError: ignored

Using df - a dataframe with smaller subset of the data

In [ ]:
df = master_df[:100000]
# df[' fare_amount']

###DATA CLEANING

In [ ]:
#apply data preprocessing
for i in range(df.shape[0]):
  # Check if the fare_amount for the current row is 0
    if df[' trip_distance'][i] > 1000.0:
      # If fare_amount is 0, set it to NaN using numpy
        df[' trip_distance'][i] = np.nan

In [ ]:
# Loop through each row in the dataframe
for i in range(df.shape[0]):
  # Check if the fare_amount value in the current row is 0
    if df[' fare_amount'][i] == 0:
      # Replace the fare_amount value in the current row with NaN
        df[' fare_amount'][i] = np.nan

In [ ]:
df.dropna()
df.shape

In [ ]:
# Check for passenger counts greater than 6 or equal to 0 and replace with NaN
for i in range(df.shape[0]):
    if df[' passenger_count'][i] > 6 or df[' passenger_count'][i] == 0:
        df[' passenger_count'][i] = np.nan

In [ ]:
# Check for unknown payment types and replace with NaN
for i in range(df.shape[0]):
    if df[' payment_type'][i] == 'UNK':
        df[' payment_type'][i] = np.nan

In [ ]:
# Check for tip amounts that are strings and replace with NaN
for i in range(df.shape[0]):
    if type(df[' tip_amount'][i]) == str:
        df[' tip_amount'][i] = np.nan

####K-Means clustering for calculating tip bins

In [ ]:
# Create a KMeans clustering model with 5 clusters
km = sklearn.cluster.KMeans(n_clusters = 5)
# Fit the KMeans model to the tip_amount values in the dataframe
km.fit(df[[' tip_amount']].values)

In [ ]:
# Assign each data point to a cluster based on the KMeans model
df['cluster_class']= km.labels_

In [ ]:
km.cluster_centers_

###UTILS FOR FEATURES

####Date Time Features

In [ ]:
# Import the datetime module to manipulate dates and times
from datetime import datetime
# Define a function to interpret the pickup time in a dataframe
def InterpretPickupTime(df, field):
    count = df.count()[0]
    month = [0]*count #1-12
    hourOfDay = [0]*count #0-23
    dayOfWeek = [0]*count #0=monday, 6=sunday
    weekend = ["X"]*count #weekday/weekend
    timegroup = ["X"]*count #0-5early, 6-10morning, 11-15afternoon, 16-20evening, 21-23night
    # Try to convert the pickup time in each row of the dataframe to a datetime object, and extract month, hour of day, day of week, weekend, and time group values
    try:
        for i in range(count):
            dt = datetime.strptime(df[field][i],"%Y-%m-%d %H:%M:%S")
            month[i] = dt.month
            dayOfWeek[i] = dt.weekday()
            hourOfDay[i] = dt.hour
            if dt.weekday()> 4:
                weekend[i] = "weekend";
            else:
                weekend[i] = "weekday";
            if hourOfDay[i] < 6:
                timegroup[i] = "early"
            elif hourOfDay[i] < 11:
                timegroup[i] = "morning"
            elif hourOfDay[i] < 16:
                timegroup[i] = "afternoon"
            elif hourOfDay[i] < 21:
                timegroup[i] = "evening"
            else:
                timegroup[i] = "night"
    # If an exception is raised (e.g. if the pickup time is not in the expected format), print an error message and return
    except:
        print "Exception##" + str(i) + "  "+ df[field][i]+ "## "
        return
    # Add new columns to the dataframe to store the extracted month, hour of day, day of week, weekend, and time group values    
    df[field+"month"] = month
    df[field+"hourOfDay"] = hourOfDay
    df[field+"dayOfWeek"] = dayOfWeek
    df[field+"weekend"] = weekend
    df[field+"timegroup"] = timegroup

####Tip Class Features

In [ ]:
# Define a function to classify tip amounts into different classes
def GetTipClasses(tip):
    if tip == 0.0:
        return "Z"
    elif tip > 0.0 and tip < 1.0:
        return "A"
    elif tip >= 1.0 and tip < 2.0:
        return "B"
    elif tip >= 2.0 and tip < 3.0:
        return "C"
    elif tip >= 3.0 and tip < 4.0:
        return "D"
    elif tip >= 4.0 and tip < 5.0:
        return "E"
    elif tip >= 5.0 and tip < 7.5:
        return "F"
    elif tip >= 7.5 and tip < 10.0:
        return "G"
    else:
        return "H"
# Define a function to calculate and assign tip classes to a dataframe
def CalculateTipClasses(df):
    count = len(df)
#     print count
    tip_df = list()
    tip_amnt = df[" tip_amount"].tolist()
    print len(tip_amnt)
    for i in range (count):
        tip_df.append(GetTipClasses(tip_amnt[i]))
#     print len(tip_df)
    df["tip_class"] = np.array(tip_df)

#### TIP Class Non zero 

In [ ]:
# Define a function to assign a letter-based tip class to a numerical tip amount
# based on a set of predetermined ranges.
def GetTipClasses_nz(tip):
    if tip > 0.0 and tip < 1.0:
        return "A"
    elif tip >= 1.0 and tip < 2.0:
        return "B"
    elif tip >= 2.0 and tip < 3.0:
        return "C"
    elif tip >= 3.0 and tip < 4.0:
        return "D"
    elif tip >= 4.0 and tip < 5.0:
        return "E"
    elif tip >= 5.0 and tip < 7.5:
        return "F"
    elif tip >= 7.5 and tip < 10.0:
        return "G"
    else:
        return "H"
# Define a function to add a column to a DataFrame containing the letter-based tip classes
# for the tip amounts in the 'tip_amount' column of the DataFrame.    
def CalculateTipClasses_nz(df):
    count = df.count()[0]
    tip_df = ['X']*(count)
    tip_amnt = df[" tip_amount"].tolist()
    # Loop over each row of the DataFrame, calculate the tip class for the corresponding tip amount,
    # and add it to the 'tip_df' list.
    for i in range (count):
        tip_df[i] = GetTipClasses_nz(tip_amnt[i])
    # Add the 'tip_class_nz' column to the DataFrame containing the calculated tip classes.
    df["tip_class_nz"] = np.array(tip_df)

####Tip no Tip Features

In [ ]:
#function to determine if  tip was given or not
def GetTipNoTip(tip):
    if tip == 0.0:
        return "NOTIP"
    else:
        return "TIP"
#function to determine if tip was given or not for all the records in the dataframe
def CalculateTipNoTip(df):
    count = df.count()[0]
#     print count
    tip_df = ['X']*count
    tip_amnt = df[" tip_amount"].tolist()
    for i in range (count):
        tip_df[i] = GetTipNoTip(tip_amnt[i])
    df["tip_no_tip"] = np.array(tip_df)

### TIP PERCENT

In [ ]:
# This function calculates the tip percentage given the tip amount and the fare amount.
def GetTipPercent(tip, fare):
        return tip*100/fare
# This function calculates the tip percentage for each row in the dataframe.
def CalcuateTipPercentage(df):
    count = df.count()[0]
    tip_df = ['X']*count
    tip_amnt = df[" tip_amount"].tolist()
    fare_amnt = df[' fare_amount'].tolist()
    # Loop through each row of the dataframe and calculate the tip percentage.
    for i in range (count):
        if tip_amnt[i] == 0 or fare_amnt[i] == 0:
            tip_df[i] = 0
        else:
            tip_df[i] = GetTipPercent(tip_amnt[i], fare_amnt[i])
    df["tip_percent"] = np.array(tip_df)   

####Zip Code Features

In [ ]:
# Function to pick the zip code with the minimum latitude that matches a given latitude and longitude
def PickZipCode(Lat,Long, zipcodes_NY):
    #first pick min latitude    
    minLong = min(zipcodes_NY.longitude.tolist(), key=lambda x:abs(x-Long))
    minLat = min(zipcodes_NY[zipcodes_NY.longitude == minLong].latitude.tolist(), key=lambda x:abs(x-Lat))
    z = zipcodes_NY[((zipcodes_NY.longitude==minLong)&(zipcodes_NY.latitude==minLat))]["zip"].tolist()[0]
    return z

# Function to assign zip codes to a given field in a data frame
def AssignZipcodes(df,field):
    count = df.count()[0]
    zipz = [0]*count
    print count
    nyc_df = pd.read_csv("..data/nycitydata.csv")
    A = nyc_df.zipcodes.tolist()
    zipcodes_NY = pd.read_csv("..data/zipcode_NYC.csv")
    zipcodes_NY = zipcodes_NY[zipcodes_NY.state=="NY"]
    zipcodes_NY.count()
    B = zipcodes_NY.zip.tolist()
    keepList = [x for x in B if x in A]
    zipcodes_NY = zipcodes_NY[zipcodes_NY.zip.isin(keepList)]
    try:
    # Iterate through each row of the data frame and assign a zip code to the corresponding field  
        for i in range(count):
            if i%10000 == 0:#test
                print i
            zipz[i] = PickZipCode(df[" "+field+"_latitude"][i],df[" "+field+"_longitude"][i], zipcodes_NY)
    except:
        print "exception "+str(i)
    df[field+"zipz"]=zipz

In [ ]:
def GetNYCityDataByZipCode():
    nyc_df = pd.read_csv("..data/nycitydata.csv")
    zipDiction={}
    
    # Loop through the rows of the data and add the data to the dictionary

    for i in range(nyc_df.count()[0]):
        zipDiction[nyc_df.zipcodes[i]] = [nyc_df.costOfLivingIndex[i],nyc_df.populationDensity[i],nyc_df.percentageMales[i],nyc_df.medianHouseholdIncome[i]]     
    return zipDiction   

In [ ]:
def AddCityData(df,field):
    # Get the number of rows in the dataframe
    count = df.count()[0]
    # Create empty lists to store the values of the city data fields
    COLI = [0]*count
    PD = [0]*count
    PM = [0]*count
    MHI = [0]*count
    # Retrieve the dictionary of city data by zip code
    diction = GetNYCityDataByZipCode()
    print count
    try:
      # Loop through each row in the dataframe
        for i in range(count):
          # Print progress every 10000 rows
            if i%10000 == 0:
                print i
            # Retrieve the city data for the zip code in the current row
            temp = diction[df[field+"zipz"][i]]
            # Store the values in the appropriate lists
            COLI[i] = temp[0]
            PD[i] = temp[1]
            PM[i] = temp[2]
            MHI[i] = temp[3]
    except:
        print "exception "+str(i)
    # Add the city data fields to the dataframe    
    df[field+"CostOfLivingIndex"]=COLI
    df[field+"PopulationDensity"]=PD
    df[field+"PercentageMale"]=PM
    df[field+"MedianHouseHoldIncome"]=MHI
    

In [ ]:
AddCityData(df,"dropoff")

###CREATE NEW FEATURES

####Time Based

In [ ]:
InterpretPickupTime(df, " pickup_datetime")
InterpretPickupTime(df, " dropoff_datetime")

####Tip Based

In [ ]:
CalculateTipClasses(df)


In [ ]:
CalculateTipNoTip(df)

In [ ]:
CalcuateTipPercentage(df)

In [ ]:
km = sklearn.cluster.KMeans(n_clusters = 5)
km.fit(df[['tip_percent']].values)

In [ ]:
km.cluster_centers_

In [ ]:
df['cluster_percent_class']= km.labels_

####Location Based

In [ ]:
AssignZipcodes(df, 'dropoff')

###MODELS

####Utils for models

In [ ]:
# This function runs Support Vector Machines (SVM) algorithm on the training data and prints the accuracy of the model
def run_SVM(X_train, Y_train, Y_dev):
    svm_clf = svm.SVC(kernel='rbf')
    svm_clf.fit(X_train, Y_train.reshape(Y_train.shape[0],))
    Y_pred_dev = svm_clf.predict(X_dev)
    SVM_acc = float(sum(map(lambda x,y : x==y, Y_pred_dev, Y_dev)))/len(Y_dev)
    print "SVM RBF Kernel set accuracy: %f" %((SVM_acc)*100)

# This function runs decision tree algorithm on the training data and prints the accuracy of the model
def run_decision_tree(X_train, Y_train, Y_dev):
    dtc_clf = tree.DecisionTreeClassifier()
    dtc_clf.fit(X_train, Y_train)
    Y_pred_dev1 = dtc_clf.predict(X_dev)
    DTC_acc = float(sum(map(lambda x,y : x==y, Y_pred_dev1, Y_dev)))/len(Y_dev)
    print "DRC set accuracy: %f" %((DTC_acc)*100)

# This function runs random forests algorithm on the training data and prints the accuracy of the model
def run_random_forests(X_train, Y_train, Y_dev):
    rf_clf = RandomForestClassifier(n_estimators=10)
    rf_clf.fit(X_train, Y_train)
    Y_pred_dev1 = rf_clf.predict(X_dev)
    DTC_acc = float(sum(map(lambda x,y : x==y, Y_pred_dev1, Y_dev)))/len(Y_dev)
    print "RF set accuracy: %f" %((DTC_acc)*100)
    
# This function runs AdaBoost algorithm on the training data and prints the accuracy of the model    
def run_Adaboost(X_train, Y_train, Y_dev):
    adb_clf = AdaBoostClassifier(base_estimator=None, n_estimators=50)
    adb_clf.fit(X_train, Y_train)
    Y_pred_dev1 = adb_clf.predict(X_dev)
    DTC_acc = float(sum(map(lambda x,y : x==y, Y_pred_dev1, Y_dev)))/len(Y_dev)
    print "Adaboost set accuracy: %f" %((DTC_acc)*100)

In [ ]:
print [(col, pd.isnull(df[col]).sum()) for col in df.columns]

In [ ]:
def GetTrainingAndTest(df):
    tr = int(math.floor(0.7*df.count()[0]))
    training = df[0:tr]
    test = df[tr:]
    return training,test

In [ ]:
feature_cols = [ ' fare_amount', ' payment_type', ' tolls_amount', 'pickupzipz','pickupCostOfLivingIndex']

#'dropoffzipz','dropoffCostOfLivingIndex',  trip_distance', ' trip_time_in_secs',
#' passenger_count', ' pickup_datetimeweekend', ' dropoff_datetimeweekend'

###Tip No-Tip Classification Model

In [ ]:
#splitting dataset into train and test sets
train_df,test_df = GetTrainingAndTest(df)
dvec = DictVectorizer()
#Converting feature columns of training set into a dictionary of key-value pairs and transforming it into an array
dict_df = train_df[feature_cols].T.to_dict().values()
train_data = pd.DataFrame(dvec.fit_transform(dict_df).toarray())
#Converting feature columns of test set into a dictionary of key-value pairs and transforming it into an array
dict_df = test_df[feature_cols].T.to_dict().values()
test_data = pd.DataFrame(dvec.transform(dict_df).toarray())
#X_train and X_dev, and the corresponding target variables to Y_train and Y_dev
X_train = train_data
Y_train =  train_df['tip_no_tip'].reshape(len(train_df['tip_no_tip']), 1)

X_dev = test_data
Y_dev = test_df['tip_no_tip'].reshape(len(test_df['tip_no_tip']), 1)

####Support Vector Machine Classifier

In [ ]:
run_SVM(X_train, Y_train, Y_dev)

####Decision Tree Classifier

In [ ]:
run_decision_tree(X_train, Y_train, Y_dev)

#### Random Forest Classifier

In [ ]:
run_random_forests(X_train, Y_train, Y_dev)

###Non zero Tip Classification Model

In [ ]:
#This code calculates tip classes for all trips in the given dataframe.
CalculateTipClasses(master_df)
#This code interprets and extracts the pickup time from the given column in the dataframe.
InterpretPickupTime(master_df, " pickup_datetime")
#This code interprets and extracts the dropoff time from the given column in the dataframe.
InterpretPickupTime(master_df, " dropoff_datetime")
#This code filters out the rows where tip amount is greater than zero from the given dataframe and takes first 100000 rows.
df_tipnz= master_df[master_df[' tip_amount'] > 0.0][:100000]
#Then it prints the shape and unique values of the tip_class column for this filtered dataframe.
print df_tipnz.shape
df.shape
print df_tipnz.tip_class.unique()

In [ ]:
#Splitting the data into training and test sets
train_df,test_df = GetTrainingAndTest(df_tipnz)

dvec = DictVectorizer()
#Extracting the features from training set and converting them into a dictionary
dict_df = train_df[feature_cols].T.to_dict().values()
#Converting the dictionary of features into an array and transforming it using DictVectorizer
train_data = pd.DataFrame(dvec.fit_transform(dict_df).toarray())
#Extracting the features from test set and converting them into a dictionary

dict_df = test_df[feature_cols].T.to_dict().values()
#Converting the dictionary of features into an array and transforming it using DictVectorizer
test_data = pd.DataFrame(dvec.transform(dict_df).toarray())
#Assigning training features and labels to variables
X_train = train_data
Y_train =  train_df['tip_class'].reshape(len(train_df['tip_class']), 1)
#Assigning test features and labels to variables
X_dev = test_data
Y_dev = test_df['tip_class'].reshape(len(test_df['tip_class']), 1)

####Support Vector Machine Classifier

In [ ]:
run_SVM(X_train, Y_train, Y_dev)

####Decision Tree Classifier

In [ ]:
run_decision_tree(X_train, Y_train, Y_dev)

###Tip Classification Model

In [ ]:
#Split the dataframe into training and testing sets
train_df,test_df = GetTrainingAndTest(df)
#Create a DictVectorizer object to convert feature dictionaries to feature arrays
dvec = DictVectorizer()
#Convert the feature dictionaries in the training set to feature arrays
dict_df = train_df[feature_cols].T.to_dict().values()
train_data = pd.DataFrame(dvec.fit_transform(dict_df).toarray())
#Convert the feature dictionaries in the testing set to feature arrays using the same DictVectorizer object as the training set
dict_df = test_df[feature_cols].T.to_dict().values()
test_data = pd.DataFrame(dvec.transform(dict_df).toarray())

X_train = train_data
Y_train =  train_df['tip_class'].reshape(len(train_df['tip_class']), 1)

X_dev = test_data
Y_dev = test_df['tip_class'].reshape(len(test_df['tip_class']), 1)

####Support Vector Machine Classifier

In [ ]:
run_SVM(X_train, Y_train, Y_dev)

####Decision Tree Classifier

In [ ]:
run_decision_tree(X_train, Y_train, Y_dev)

### TIP PERCENT CLASSIFIFCATION MODEL

In [ ]:
run_decision_tree(X_train, Y_train, Y_dev)

In [ ]:
#Split the dataframe into training and testing sets
train_df,test_df = GetTrainingAndTest(df)
#Create a DictVectorizer object to convert feature dictionaries to feature arrays
dvec = DictVectorizer()
#Convert the feature dictionaries in the training set to feature arrays
dict_df = train_df[feature_cols].T.to_dict().values()
train_data = pd.DataFrame(dvec.fit_transform(dict_df).toarray())
#Convert the feature dictionaries in the testing set to feature arrays using the same DictVectorizer object as the training set
dict_df = test_df[feature_cols].T.to_dict().values()
test_data = pd.DataFrame(dvec.transform(dict_df).toarray())

X_train = train_data
Y_train =  train_df['cluster_percent_class'].reshape(len(train_df['cluster_percent_class']), 1)

X_dev = test_data
Y_dev = test_df['cluster_percent_class'].reshape(len(test_df['cluster_percent_class']), 1)

###Tip Amount Prediction Regression Model

In [ ]:
#Split the dataset into training and test dataframes
train_df,test_df = GetTrainingAndTest(df)
#Convert the categorical features to numeric using DictVectorizer
dvec = DictVectorizer()
dict_df = train_df[feature_cols].T.to_dict().values()
train_data = pd.DataFrame(dvec.fit_transform(dict_df).toarray())

dict_df = test_df[feature_cols].T.to_dict().values()
test_data = pd.DataFrame(dvec.transform(dict_df).toarray())

X_train = train_data
Y_train =  train_df[' tip_amount'].reshape(len(train_df[' tip_amount']), 1)

X_dev = test_data
Y_dev = test_df[' tip_amount'].reshape(len(test_df[' tip_amount']), 1)

####Baseline Accuracy for Regression

In [ ]:
#Compute the Mean Absolute Error (MAE) of tip amount compared to the mean tip amount
tip_mean = df[' tip_amount'].mean()

diff_df = df[' tip_amount'].apply(lambda(x): abs(x-tip_mean))
mae = diff_df.sum()/df.shape[0]
print "Mean absolute Error: %f" %(mae)

####Linear Regression Model

In [ ]:
#Extracting the tip amount as the target variable for regression
Y_reg_train =  train_df[' tip_amount'].reshape(len(train_df[' tip_amount']), 1)
Y_reg_dev = test_df[' tip_amount'].reshape(len(test_df[' tip_amount']), 1)
#Initializing a linear regression model and fitting it on the training data
lr = linear_model.LinearRegression()
lr.fit(X_train, Y_reg_train.reshape(Y_train.shape[0],))
#Predicting tip amounts for the test set and evaluating the model's accuracy
Y_pred_dev = lr.predict(X_dev)
Y_pred_dev = Y_pred_dev.reshape(len(Y_pred_dev), 1)
print "Linear Regression Accuracy:%f" %(lr.score(X_dev, Y_reg_dev))
#Calculating the mean absolute error of the model
mae = np.sum(np.absolute(Y_reg_dev - Y_pred_dev))/Y_reg_dev.shape[0]
print "Mean absolute Error: %f" %(mae)

####SVM Regression Model

In [ ]:
# Fit a support vector regression model to the training data

svr = svm.SVR()
svr.fit(X_train, Y_reg_train.reshape(Y_train.shape[0],))
# Evaluate the model on the test data and print the R^2 score
# This score represents the proportion of the variance in the target variable explained by the model
print svr.score(X_dev, Y_reg_dev)

####Lasso Regression Model

In [ ]:
lasso = linear_model.Lasso()
lasso.fit(X_train, Y_reg_train.reshape(Y_train.shape[0],))
Y_pred_dev = lasso.predict(X_dev).reshape(len(Y_pred_dev), 1)
print "Lasso Regression Accuracy:%f" %(lasso.score(X_dev, Y_reg_dev))

mae = np.sum(np.absolute(Y_reg_dev - Y_pred_dev))/Y_reg_dev.shape[0]
print "Mean absolute Error: %f" %(mae)

# print lasso.coefs